In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
import seaborn as sns

%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

from data_utils import *

plt.rcParams['figure.figsize'] = 7, 3

# Range-only datasets for localization 

### Download .mat files and save them in folder ./datasets/

WiFi: http://www.robesafe.es/repository/UAHWiFiDataset/

Landmower: https://github.com/gtrll/gpslam/tree/master/matlab/data

See README for file description. 

## Choose dataset

In [ ]:
anchor_names = None  # use all anchors by default.

#filename = 'datasets/uah1.mat'
# fingers. works ok.
#filename = 'datasets/Plaza1.mat'; # zig zag. does not work super well.
filename = 'datasets/Plaza2.mat' # triangle. works well.

verbose = False
period_it = 10
traj = get_trajectory(filename)
dataname = filename.split('/')[-1].split('.')[0]

if dataname == 'uah1':
    t_window = 1.0
    eps = 2.0
    xlim = 0, 50
    ylim = -20, 20

    min_time = 0
    max_time = 1000

    # for iterative.
    n_complexity_it = 2
    model_it = 'polynomial'
    t_window_it = 80

elif dataname == 'Plaza1':
    t_window = 0.5
    eps = 0.5
    xlim = -50, 10
    ylim = -20, 75

    # choose one:
    min_time = 0  # first big circle
    max_time = 200  # first big circle
    min_time = 510  # first loop
    max_time = 600  # first loop
    min_time = 0  # first few loops
    max_time = 1000  # first few loops.

    # for iterative.
    n_complexity_it = 3
    model_it = 'full_bandlimited'
    period_it = 40
    t_window_it = 20

elif dataname == 'Plaza2':
    t_window = 0.1
    eps = 0.2
    xlim = -80, 10
    ylim = -20, 75

    min_time = 45.1
    period = 101 - 45
    print('period:', period)
    num_loops = 2
    max_time = min_time + num_loops * period
    traj.period = period

    # for iterative.
    n_complexity_it = 3
    model_it = 'bandlimited'
    period_it = 40
    t_window_it = 40
    
    #anchor_names = ['Range {}'.format(i) for i in range(1, 4)]

result_dict = loadmat(filename)

## Prepare dataset

In [ ]:
anchor_data = result_dict['TL']
range_data = result_dict['TD']
gt_data = result_dict['GT']

anchors_df = create_anchors_df(anchor_data)
anchors_df = format_anchors_df(anchors_df, range_system_id=range_system_id, gt_system_id=gt_system_id)

full_df = create_full_df(range_data, gt_data)
full_df = format_data_df(full_df, anchors_df, gt_system_id=gt_system_id, range_system_id=range_system_id)
if verbose:
    print('time going from {:.1f} to {:.1f}'.format(full_df.timestamp.min(), full_df.timestamp.max()))
full_df = full_df[(full_df.timestamp >= min_time) & (full_df.timestamp <= max_time)]
full_df.loc[:, 'timestamp'] = full_df.timestamp - full_df.timestamp.min()

fig, axs = plt.subplots(1, 2)
sns.scatterplot(data=full_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=full_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, ax=axs[1])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')

print('adding ground truth...')
full_df = add_gt_raw(full_df, t_window=t_window, gt_system_id=gt_system_id)
full_df.loc[:, "distance_gt"] = full_df.apply(
    lambda row: apply_distance_gt(row, anchors_df, gt_system_id=gt_system_id), axis=1)
print('...done')

## Plot distance measurements

In [ ]:
fig, axs = plt.subplots(1, 2)
range_df = full_df.loc[full_df.system_id == range_system_id].copy()
sns.scatterplot(data=range_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')
for a_id, px, py in zip(anchors_df.anchor_name, anchors_df.px, anchors_df.py):
    print(a_id, px, py)
    axs[0].annotate(s=a_id, xy=(px+2,py+2), color='red')
axs[0].legend('')
sns.scatterplot(data=range_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, ax=axs[1])

plot_distance_times(full_df)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5, 2)
ax = plot_distance_errors(full_df, ax=ax)
#ax.set_title(dataname + ' data set')
savefig(fig, 'results/accuracy.pdf')

In [ ]:
plt.figure()
range_df.loc[:, 'distance_error'] = range_df.distance.values - range_df.distance_gt.values
sns.scatterplot(data=range_df, x='px', y='py', hue='distance_error', linewidth=0.0)

In [ ]:
plt.figure()
sns.scatterplot(data=range_df, x='px', y='py', hue='anchor_name', linewidth=0.0)

## Filter measurements

In [ ]:
filtered_df = full_df[full_df.distance<=50]

plot_distance_times(filtered_df)
ax = plot_distance_errors(filtered_df)

## Construct parameters.

In [ ]:
from evaluate_dataset import compute_distance_matrix

chosen_df = full_df
#chosen_df = filtered_df
chosen_distance = 'distance'
#chosen_distance = 'distance_gt'

## Construct anchors. 
if anchor_names is None:
    anchors = anchors_df.loc[:, ['px', 'py', 'pz']].values.astype(np.float32).T
else:
    anchors_df = anchors_df.loc[anchors_df.anchor_name.isin(anchor_names)]
    anchors = get_coordinates(anchors_df, anchor_names)

## Construct times.
range_df = chosen_df[chosen_df.system_id == range_system_id]
times = range_df.timestamp.unique()

## Construct D.
D, times = compute_distance_matrix(chosen_df, anchors_df, anchor_names, times, chosen_distance)
if np.sum(D > 0) > D.shape[0]:
    print('Warning: multiple measurements for times:{}/{}!'.format(
          np.sum(np.sum(D > 0, axis=1)>1), D.shape[0]))

## Construct ground truth.
ground_truth_pos = get_ground_truth(chosen_df, times)

# Global algorithm

In [ ]:
list_complexities = [3, 5, 21, 51]

fig_size = [8, 2]
ylim = [-25, 70]

fig, axs = plt.subplots(1, len(list_complexities), sharex=True, sharey=True)
fig.set_size_inches(*fig_size) 
for ax, n_complexity in zip(axs, list_complexities):

    traj.set_n_complexity(n_complexity)
    basis = traj.get_basis(times=times)

    Chat_weighted = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=True)
    Chat = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=False)

    traj.set_coeffs(coeffs=Chat)

    traj_weighted = traj.copy()
    traj_weighted.set_coeffs(coeffs=Chat_weighted)

    traj.plot_pretty(times=times, color="C0", label='non-weighted', ax=ax)
    traj_weighted.plot_pretty(times=times, color='C1', label='weighted', ax=ax)
    ax.plot(full_df.px, full_df.py, color='black', label='ground truth')
    ax.set_xlabel('x [m]')
    ax.set_title('K = {}'.format(traj.n_complexity))
    
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    #ax.axis('equal')
#ax.axis('equal')
axs[0].set_ylabel('y [m]')
legend = ax.legend(loc='lower right', ncol=3, facecolor='white', framealpha=1)
savefig(fig, 'results/results.pdf')

In [ ]:
import hypothesis as h

# remove some measurements. 
n_complexity = 5
traj.set_n_complexity(n_complexity)

num_seeds = 3
min_number = n_complexity*(traj.dim + 2) - 1 
print(min_number, D.shape[0])
#n_measurements_list = np.arange(D.shape[0], min_number, step=-100)
#n_measurements_list = [19, 20, 30, 40, 50, 100, 200, 300, 499]
n_measurements_list = [20, 30, 50, 200][::-1]

alpha = 0.2
fig, axs = plt.subplots(1, len(n_measurements_list), sharex=True, sharey=True)
fig.set_size_inches(*fig_size) 

for ax, n_measurements in zip(axs, n_measurements_list):
    
    label_weight = 'weighted'
    label = 'non-weighted'
    
    coeffs = np.empty([traj.dim, n_complexity, 0])
    coeffs_weighted = np.empty([traj.dim, n_complexity, 0])
    for seed in range(num_seeds):
        np.random.seed(seed)
        indices = np.random.choice(D.shape[0], n_measurements, replace=False)

        D_small = D[indices, :]
        mask = (D_small > 0).astype(np.float)
        
        p = np.sort(np.sum(mask, axis=0))[::-1]
        if not h.limit_condition(list(p), traj.dim + 1, n_complexity):
            print("insufficient rank")
        
        times_small = np.array(times)[indices]

        basis_small = traj.get_basis(times=times_small)

        Chat_weighted = alternativePseudoInverse(D_small, anchors[:2, :], basis_small, weighted=True)
        Chat = alternativePseudoInverse(D_small, anchors[:2, :], basis_small, weighted=False)

        coeffs = np.dstack([coeffs, Chat])
        coeffs_weighted = np.dstack([coeffs_weighted, Chat_weighted])
        
        traj.set_coeffs(coeffs=Chat)

        traj_weighted = traj.copy()
        traj_weighted.set_coeffs(coeffs=Chat_weighted)

        traj.plot_pretty(times=times, color='C0', ax=ax, alpha=alpha)
        traj_weighted.plot_pretty(times=times, color='C1', ax=ax, alpha=alpha)
        
    Chat_avg = np.mean(coeffs, axis=2)
    Chat_weighted_avg = np.mean(coeffs_weighted, axis=2)
    traj.set_coeffs(coeffs=Chat_avg)
    traj_weighted.set_coeffs(coeffs=Chat_weighted_avg)
    traj.plot_pretty(times=times, color='C0', label=label, ax=ax)
    traj_weighted.plot_pretty(times=times, color='C1', label=label_weight, ax=ax)
        
    ax.plot(full_df.px, full_df.py, color='black', label='ground truth')
    ax.set_xlabel('x [m]')
    ax.set_title('N={}'.format(n_measurements))
    
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)

axs[0].set_ylabel('y [m]')
legend = ax.legend(loc='lower right', ncol=3, facecolor='white', framealpha=1)
savefig(fig, 'results/downsampling.pdf')

# Compare with other algorithms

In [ ]:
def get_anchors_and_distances(D, idx):
    """
    Given squared distance matrix D and time index idx, 
    find all latest distance measurements.
    """
    r2 = []
    anchors = []
    for a_id in range(D.shape[1]): 
        indices = np.where(D[:idx, a_id] > 0)[0]
        if len(indices) > 0:
            latest_idx = indices[-1]
            r2.append(D[latest_idx, a_id])
            anchors.append(a_id)
    return np.array(r2).reshape((-1, 1)), anchors

def calculate_error(Chat, C, error_type='MAE'):
    if error_type == 'MAE':
        return np.mean(np.abs(Chat - C))

In [ ]:
def cost_function(C_k_vec, D, A, F, verbose=False):
    """ 
    :param C_k: dim x K
    :param F: K x N
    :param A: dim x M
    :param D: N x M
    """
    C_k = C_k_vec.reshape((-1, 2)).T
    R = C_k.dot(F)
    if verbose:
        print('R  dim x N', R.shape) # dim x N
    diff = R[:, :, None] - A[:, None, :]
    if verbose:
        print('diff  dim x N x M', diff.shape)
    D_est = np.linalg.norm(diff, axis=0)**2 #  dim x N x M
    if verbose:
        print('N x M', D_est.shape, D.shape)
    return np.power(D_est - D, 2).reshape((-1,))
    
from measurements import get_measurements
    

n_complexity = 5
traj.set_n_complexity(n_complexity)
basis = traj.get_basis(times=times)
traj.set_coeffs(1)

#basis, D = get_measurements(traj, anchors[:2, :], times=times)
Chat = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=False)

print('OURS error', calculate_error(Chat, traj.coeffs))

from scipy.optimize import root
from scipy.optimize import least_squares

#root(cost_function, x0=Chat.reshape((-1,)), method='lm', args=(D, anchors[:2], basis))

res = least_squares(cost_function, x0=Chat.reshape((-1,)), method='lm', args=(D, anchors[:2], basis))
Cref = res.x.reshape((2, -1))

x0 = np.random.rand(*Chat.shape)
res = least_squares(cost_function, x0=x0.reshape((-1,)), method='lm', args=(D, anchors[:2], basis))
Crand = res.x.reshape((2, -1))
print('LM refined error', calculate_error(Cref, traj.coeffs))
print('LM random error', calculate_error(Crand, traj.coeffs))

In [ ]:
#from pylocus.basics import get_edm
from pylocus.lateration import SRLS
import time

n_measurements_list = [20, 30, 50, 200][::-1]

methods = ['srls'] #['non-weighted', 'srls', 'edm']
num_seeds = 1

for ax, n_measurements in zip(axs, n_measurements_list):
    coeffs = np.empty([traj.dim, n_complexity, 0])
    
    for seed in range(num_seeds):
        fig, ax = plt.subplots()
        
        np.random.seed(seed)
        indices = np.random.choice(D.shape[0], n_measurements, replace=False)

        D_small = D[indices, :]
        mask = (D_small > 0).astype(np.float)
        times_small = np.array(times)[indices]
        
        for method in methods: 
            
            t1 = time.time()
            if method == 'weighted':
                basis = traj.get_basis(times=times_small)
                Chat = alternativePseudoInverse(D_small, anchors[:2, :], basis, weighted=True)
                traj.set_coeffs(coeffs=Chat)
                traj.plot_pretty(times=times, color='C0', ax=ax, alpha=alpha, label=method)
            elif method == 'non-weighted':
                basis = traj.get_basis(times=times_small)
                Chat = alternativePseudoInverse(D_small, anchors[:2, :], basis, weighted=True)
                traj.set_coeffs(coeffs=Chat)
                traj.plot_pretty(times=times, color='C1', ax=ax, alpha=alpha, label=method)
            elif method == 'srls':
                label = 'SRLS'
                for idx in indices[::traj.dim+1]:
                    r2, a_indices = get_anchors_and_distances(D, idx)
                    if len(r2) > traj.dim + 1:
                        anchors_srls = anchors[:2, a_indices].T#N x d
                        weights = np.ones(r2.shape)
                        x = SRLS(anchors_srls, weights, r2)
                        ax.scatter(*x, color='C2', label=label)
                        label=None
                    else:
                        print('skipping')
            t2 = time.time()
    ax.plot(full_df.px, full_df.py, color='black', label='ground truth')
    ax.set_xlabel('x [m]')
    ax.set_title('N={}'.format(n_measurements))
    
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)

axs[0].set_ylabel('y [m]')
legend = ax.legend(loc='lower right', ncol=3, facecolor='white', framealpha=1)
savefig(fig, 'results/others.pdf')

# Iterative algorithms

In [ ]:
traj_it = traj.copy()
traj_it.set_n_complexity(n_complexity_it)
traj_it.model = model_it
traj_it.period = period_it
basis = traj_it.get_basis(times=times)
print('Using trajectory model: \n model={}, K={}, period={}'.format(traj_it.model, traj_it.n_complexity, traj_it.period))

In [ ]:
### Averaging algorithm
print('averaging with time window', t_window_it)
C_list, t_list = averaging_algorithm(D, anchors[:2, :], basis, times, t_window=t_window_it)
ax1 = plot_individual(C_list, t_list, traj_it)
ax1.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]

In [ ]:
### Build up algorithm
C_list, t_list = build_up_algorithm(D, anchors[:2, :], basis, times, eps=eps, verbose=False)
ax1 = plot_individual(C_list, t_list, traj_it.copy())
ax1.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')

result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]